In [12]:
# RAG Setup: loaders, splitter, embeddings, vector store
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# 1) Load your PDFs
pdf_paths = [
    "../query6_data/Asset allocation.pdf"
]

docs = []
for p in pdf_paths:
    loader = PyPDFLoader(p)
    # Each page comes with metadata; loader returns Document objects
    docs.extend(loader.load())

# 2) Split into chunks (keep overlaps for context continuity)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=900, chunk_overlap=150, separators=["\n\n", "\n", " ", ""]
)
splits = splitter.split_documents(docs)

# 3) Embeddings (no API key needed)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4) Build FAISS index
vecstore = FAISS.from_documents(splits, embeddings)

# 5) Create a retriever (top_k configurable inside the tool)
retriever = vecstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})


Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)


In [13]:
# Define your query
query = "Should I allocate more to equity or debt in my portfolio if I’m planning to buy a house in 10 years?"

# Search for relevant chunks
retrieved_docs = vecstore.similarity_search(query, k=3)

# Combine retrieved content
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

print(context)


Recommended Asset Allocation (General Rule of Thumb)
This is based on the age-old 100 minus age rule, adjusted for your goal rather than your age.
Why Equity Should Dominate
Pros:
• Historically, Indian equity returns average 12–15% p.a. over 10 years.
• Ideal for longer horizons: gives you time to ride out market volatility.
• Helps build a bigger corpus faster than debt instruments.
How to Invest:
• SIP in diversiﬁed equity mutual funds (e.g. large-cap, ﬂexi-cap, ELSS if tax saving).
• Consider index funds or ETFs if you want lower cost.
Why Debt Still Matters
Role:
• Adds stability to your portfolio.
• Helps protect capital as you near the goal.
• Rebalances volatility during market downturns.
How to Invest:
• Debt mutual funds (short-term, target maturity funds).
• PPF, EPF (if available), ﬁxed deposits for part of the amount.
Glide Path Strategy (Dynamic Rebalancing)

This ensures your house down payment is protected from late-stage market crashes.
Quick Illustration (₹10,000/mont